## Create Synthetic data 

<h5>For generate the data we gonna use the SDV or Synthetic Data Vault. SVD generates synthetic data by applying mathematical techniques and machine learning models such as the deep learning model.</h5>
<b>Even if the data contain multiple data types and missing data, SDV will handle it, so we only need to provide the data.

To use the SDMetrics library, you’ll need:

-Real data, loaded as a pandas DataFrame <br>
-Synthetic data, loaded as a pandas DataFrame <br>
-Metadata, represented as a dictionary format

In [1]:
import pandas as pd
import os 
import numpy as np
from sdv.tabular import CTGAN
from sdmetrics.reports.single_table import QualityReport
import warnings
warnings.filterwarnings('ignore')

/Users/tomasferreira/opt/anaconda3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
infringement_path = os.path.join('2023_SP_Dataset_v2', 'infringement_dataset_v2.csv')
description_path = os.path.join('2023_SP_Dataset_v2', 'columns_description.csv')

<h4>Now we gonna drop unnecessary columns</h4>

In [10]:
data = pd.read_csv(infringement_path)
data = data.head(1000)
data

,loan_id,infringed,contract_type,gender,has_own_car,has_own_realty,num_children,annual_income,credit_amount,credit_annuity,...,first_name,last_name,past_avg_amount_annuity,past_avg_amt_application,past_avg_amt_credit,past_loans_approved,past_loans_refused,past_loans_canceled,past_loans_unused,past_loans_total
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,Robert,Watkins,9251.775000,179055.000000,179055.000000,1.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,Jane,Navarro,56553.990000,435436.500000,484191.000000,3.0,0.0,0.0,0.0,3.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,David,Seagraves,5357.250000,24282.000000,20106.000000,1.0,0.0,0.0,0.0,1.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,Deborah,Tandy,23651.175000,272203.260000,291695.500000,5.0,3.0,1.0,0.0,9.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,David,Walker,12278.805000,150530.250000,166638.750000,6.0,0.0,0.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,101152,0,Cash loans,F,N,N,0,112500.0,495985.5,17946.0,...,Helen,Quinn,14697.306818,131739.600000,165087.000000,7.0,4.0,4.0,0.0,15.0
996,101153,0,Cash loans,F,N,Y,0,225000.0,1113840.0,57001.5,...,Laura,Todd,12175.020000,60660.000000,41850.000000,1.0,0.0,0.0,0.0,1.0
997,101154,0,Cash loans,F,Y,Y,0,144000.0,517536.0,28206.0,...,Carol,Lee,6651.360000,58181.142857,68409.642857,3.0,2.0,0.0,2.0,7.0
998,101155,0,Cash loans,M,N,Y,0,315000.0,1288350.0,37800.0,...,Greg,Barnwell,5000.877000,41513.785714,38557.285714,3.0,1.0,3.0,0.0,7.0


In [7]:
to_drop=["address",
"appendix_a", 
"appendix_b", 
"appendix_c",
"appendix_d",
"appendix_e",  
"appendix_f",  
"appendix_g",  
"appendix_h",  
"appendix_i",  
"appendix_j",  
"appendix k",  
"appendix_l",  
"appendix_m",  
"appendix n",  
"appendix_o",
"appendix_p",
"appendix q",  
"appendix_r",  
"appendix_s",  
"appendix t",  
"car_age",
"first_name",
"last_name",
"num_req_bureau_day",
"num_req_bureau_hour", 
"num_req_bureau_month", 
"num_req_bureau_qrt", 
"num_req_bureau_week", 
"num_req_bureau_year", 
"provided_email", 
"provided_homephone",
"provided_mobilephone",
"provided_workphone",
"Region_rate",
"score_ext_1",
"score_ext_2",
"score_ext_3"]

In [8]:
data.drop(to_drop, axis=1)
data

KeyError: "['address', 'appendix_a', 'appendix_b', 'appendix_c', 'appendix_d', 'appendix_e', 'appendix_f', 'appendix_g', 'appendix_h', 'appendix_i', 'appendix_j', 'appendix k', 'appendix_l', 'appendix_m', 'appendix n', 'appendix_o', 'appendix_p', 'appendix q', 'appendix_r', 'appendix_s', 'appendix t', 'Region_rate'] not found in axis"

In [11]:
model = CTGAN(epochs=15, generator_dim=(256, 256), discriminator_dim=(256, 256), verbose=True)
model.fit(data)

/Users/tomasferreira/opt/anaconda3/lib/python3.8/site-packages/rdt/transformers/categorical.py:380: UserWarning: The data contains 1 new categories that were not seen in the original data (examples: {nan}). Creating a vector of all 0s. If you want to model new categories, please fit the transformer again with the new data.
  warnings.warn(


Epoch 1, Loss G:  1.1658,Loss D: -0.1018
Epoch 2, Loss G:  1.4178,Loss D: -0.3899
Epoch 3, Loss G:  1.6833,Loss D: -0.7472
Epoch 4, Loss G:  1.7306,Loss D: -1.1818
Epoch 5, Loss G:  1.7328,Loss D: -1.5254
Epoch 6, Loss G:  1.6161,Loss D: -1.6365
Epoch 7, Loss G:  1.6246,Loss D: -1.5583
Epoch 8, Loss G:  1.4442,Loss D: -1.5703
Epoch 9, Loss G:  1.1137,Loss D: -1.1329
Epoch 10, Loss G:  1.0056,Loss D: -0.9272
Epoch 11, Loss G:  0.8041,Loss D: -0.9062
Epoch 12, Loss G:  0.7346,Loss D: -0.6505
Epoch 13, Loss G:  0.9806,Loss D: -0.4920
Epoch 14, Loss G:  0.6973,Loss D: -0.1845
Epoch 15, Loss G:  0.7280,Loss D: -0.1537


<h4>After fitting the model we gonna use it to generate the new data</h4>

In [12]:
synthetic_data = model.sample(num_rows=data.shape[0])
synthetic_data

,loan_id,infringed,contract_type,gender,has_own_car,has_own_realty,num_children,annual_income,credit_amount,credit_annuity,...,first_name,last_name,past_avg_amount_annuity,past_avg_amt_application,past_avg_amt_credit,past_loans_approved,past_loans_refused,past_loans_canceled,past_loans_unused,past_loans_total
0,100799,0,Cash loans,F,Y,Y,0,547492.307,369261.0,9243.1,...,Lucretia,Dees,NaN,66841.0,139778.0,3.0,0.0,0.0,0.0,3.0
1,101024,0,Cash loans,F,N,Y,1,143096.945,214153.0,20213.4,...,Karina,Mccorkle,NaN,177231.0,122328.0,NaN,1.0,NaN,0.0,1.0
2,100604,0,Cash loans,F,N,Y,0,457230.425,717279.0,19109.9,...,Peter,Naylor,11592.0,107639.0,141067.0,3.0,0.0,0.0,0.0,1.0
3,100620,0,Cash loans,M,N,N,2,135134.511,1169781.0,9000.8,...,Andrea,Hammock,11863.0,199957.0,120455.0,8.0,1.0,0.0,0.0,1.0
4,100948,0,Cash loans,F,Y,Y,0,186460.862,270150.0,11843.6,...,Marguerite,Hodgson,8132.0,358520.0,106948.0,3.0,NaN,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,100303,0,Cash loans,M,N,Y,0,159452.413,1727323.0,4504.5,...,Molly,Humphrey,3983.0,96238.0,161521.0,4.0,0.0,0.0,NaN,1.0
996,100348,0,Cash loans,M,Y,N,0,485351.163,660946.0,12724.9,...,Alfred,Quinn,1566.0,301054.0,90747.0,3.0,0.0,0.0,0.0,2.0
997,100159,0,Cash loans,M,N,N,0,180785.808,1182864.0,4504.5,...,Thelma,Mcbeth,10530.0,169424.0,35900.0,1.0,0.0,0.0,0.0,NaN
998,100743,0,Cash loans,M,N,Y,0,308373.694,690639.0,12816.8,...,Juliette,Berrios,5945.0,79108.0,48555.0,2.0,1.0,0.0,0.0,NaN


<h4>Computing performance score</h4>

This report evaluates the shapes of the columns (marginal distributions) and the pairwise trends between the columns (correlations).

In [13]:
report = QualityReport()

report.generate(data, synthetic_data,model.get_metadata().to_dict())

Creating report: 100%|██████████| 4/4 [00:07<00:00,  1.77s/it]



Overall Quality Score: 82.44%

Properties:
Column Shapes: 83.24%
Column Pair Trends: 81.64%


In [14]:
report.get_details(property_name='Column Shapes')

,Column,Metric,Quality Score
0,loan_id,KSComplement,0.919000
1,infringed,KSComplement,0.974000
2,num_children,KSComplement,0.876000
3,annual_income,KSComplement,0.771000
4,credit_amount,KSComplement,0.869000
5,credit_annuity,KSComplement,0.596000
6,goods_valuation,KSComplement,0.550116
7,age,KSComplement,0.971000
8,days_employed,KSComplement,0.712000
9,car_age,KSComplement,0.496186


In [15]:
report.get_visualization(property_name='Column Pair Trends')

<h4><b>Result analysis</b></h4><br>
By observing the results, we can see that the overall quality score of our new dataset is 81%. 
This result is good, but we can do a deeper analysis and see the scores on the individual columns and between the correlation in the columns. The result is basically the same, so we can conclude that our synthetic data is good. 

<b>[Pros and Cons of synthetic data](https://www.analyticssteps.com/blogs/what-synthetic-data-types-advantages-and-disadvantages)</b><br><br>
<b>Advantages:</b><br><br>
<ul>
  <li><b>Data quality</b> - Higher data quality, balance, and variety are ensured with synthetic data. Artificially created data can apply labels and automatically fill in missing quantities, allowing for more precise prediction;<br><br></li>
  <li><b> Scalability</b> - Synthetic data is used to cover the gaps left by real-world data;<br><br></li>
  <li><b> Utilization simplicity</b> - Synthetic data guarantees ‌all data has a consistent format and labelling, getting rid of errors and duplicates.<br><br></li>
</ul>
<b> Disadvantages:</b><br><br>
<ul>
  <li>Outliers are challenging to map because synthetic data merely approximates real-world data, it is not a duplicate.Therefore, some outliers that are present in original data may not be covered by synthetic data;<br><br></li>
  <li>The quality of the model depends on the data source.<br><br></li>
</ul>

